# **SILVER PREVENÇÃO** 

## CONFIGURAÇÃO DO PIPELINE

In [0]:
%sql

use catalog hive_metastore;
use schema silver;

## BIOMETRY

In [0]:
%sql

drop table if exists silver.biometria;
create table if not exists silver.biometria as (
  with 

    base as (
      select
        cast(b.session_id as int)   id_sessao,
        cast(b.event_dt as date)    dt_evento,
        lower(trim(b.status))       status_orig,
        cast(b.driver_id as int)    id_motorista,
        lower(trim(b.action))       acao_orig 
      from
        bronze.biometry b
    )

  select
    b.id_sessao,
    b.dt_evento,
    b.status_orig,
    case
      when b.status_orig = "provider_failed"  then "falha no provedor"
      when b.status_orig = "match"            then "combina"
      when b.status_orig = "not_match"         then "nao combina"
    end status,
    b.id_motorista,
    b.acao_orig,
    if(
      b.acao_orig = "fluxo_stack",
      "necessario acao",
      "sem necessidade de acao"
    ) acao,
    if(
      b.acao_orig = "fluxo_stack",
      1,
      0
    ) flg_necessario_acao
  from
    base b
)

## BIOMETRY_EXECUTION

In [0]:
%sql

drop table if exists silver.biometria_tentativas;
create table if not exists silver.biometria_tentativas as (
  with 

    base as (
      select
        cast(be.event_dt as date)      dt_evento,
        cast(be.session_id as int)     id_sessao,
        cast(be.similarity as double)  percent_similaridade,
        lower(trim(be.status))         status_orig,
        lower(trim(be.provider))       provedor
      from
        bronze.biometry_execution be
    )

  select
    b.dt_evento,
    b.id_sessao,
    b.percent_similaridade,
    b.status_orig,
    case
      when b.status_orig = "provider_failed"  then "falha no provedor"
      when b.status_orig = "match"            then "combina"
      when b.status_orig = "not_match"        then "nao combina"
    end status,
    b.provedor
  from
    base b
)


## DRIVERS

In [0]:
%sql

drop table if exists silver.motoristas;
create table if not exists silver.motoristas as (
select
  lower(trim(d.driver_id))    id_motorista,
  lower(trim(d.category))     categoria,
  lower(trim(d.modal))        modal,
  cast(d.register_dt as date) dt_registro,
  d.device_id                 id_device
from
  bronze.drivers d
)

## ORDERS

In [0]:
%sql

drop table if exists silver.ordens_servico;
create table if not exists silver.ordens_servico as (
with 

  base as (
    select
      cast(o.order_dt as date)        dt_ordem,
      cast(o.order_id as bigint)      id_ordem,
      cast(o.driver_id as int)        id_motorista,
      cast(o.order_value as double)   vlr_ordem,
      lower(trim(o.order_status))     status_ordem_orig
    from
      bronze.orders o
  )

select
  b.dt_ordem,
  b.id_ordem,
  b.id_motorista,
  b.vlr_ordem,
  b.status_ordem_orig,
  case
    when b.status_ordem_orig = "#n/d"      then "nao conhecido"
    when b.status_ordem_orig = "cancelled" then "cancelado"
    when b.status_ordem_orig = "concluded" then "concluido"
  end status_ordem
from
  base b
)